In [1]:
import AdventOfCode

In [2]:
input <- day 22

In [3]:
{-# LANGUAGE OverloadedStrings #-}

import qualified Data.Attoparsec.ByteString.Char8 as A

parseInput = do
    depth <- "depth: " *> A.decimal <* A.endOfLine
    target <- "target: " *> ((,) <$> A.decimal <*> ("," *> A.decimal))
    pure (depth, target)

(depth, target) = parsed parseInput input

In [4]:
import Data.Function.Memoize

geologicIndex :: (Int, Int) -> Int
geologicIndex = memoize $ \c -> case c of
    (0,0) -> 0
    t | t == target -> 0
    (x,0) -> x*16807
    (0,y) -> y*48271
    (x,y) -> erosionLevel (x-1,y) * erosionLevel (x,y-1)
    
erosionLevel = memoize $ \c -> (geologicIndex c + depth) `mod` 20183

riskLevel = memoize $ \c -> erosionLevel c `mod` 3 

In [5]:
import Data.List

sum $ map riskLevel [(x,y) | let (maxX,maxY) = target, x <- [0..maxX], y <- [0..maxY]]

11359

In [6]:
import qualified Data.Set as Set

data Equipment = Torch | ClimbingGear | Neither deriving (Eq, Show, Ord)

data CaveState
    = CaveState
    { minutesSoFar :: Int
    , current :: (Int, Int)
    , equipped :: Equipment
    } deriving (Eq, Show, Ord)

data Measure a = Measure { measure :: Int, value :: a } deriving (Eq, Show, Ord)

distanceFrom (x,y) (a,b) = abs (x-a) + abs (y-b)

changeEquipment :: CaveState -> CaveState
changeEquipment cs@(CaveState mins curr equip) = case riskLevel curr of
    0 -> case equip of
        ClimbingGear -> CaveState (mins+7) curr Torch
        Torch -> CaveState (mins+7) curr ClimbingGear
    1 -> case equip of
        ClimbingGear -> CaveState (mins+7) curr Neither
        Neither -> CaveState (mins+7) curr ClimbingGear
    2 -> case equip of
        Neither -> CaveState (mins+7) curr Torch
        Torch -> CaveState (mins+7) curr Neither

moves :: CaveState -> [CaveState]
moves cs@(CaveState mins c@(x,y) equip) = let
    possible = filter (\(a,b) -> a >= 0 && a < 21 && b >= 0) $ [(x-1,y), (x+1,y), (x,y-1), (x,y+1)]
    in (changeEquipment cs):(map (transition cs) possible)

transition :: CaveState -> (Int, Int) -> CaveState
transition cs@(CaveState mins curr equip) coord = case riskLevel coord of
    0 -> case equip of
        c | c `elem` [ClimbingGear, Torch] -> CaveState (mins+1) coord equip
        _ -> transition (changeEquipment cs) coord
    1 -> case equip of
        c | c `elem` [ClimbingGear, Neither] -> CaveState (mins+1) coord equip
        _ -> transition (changeEquipment cs) coord
    2 -> case equip of
        c | c `elem` [Neither, Torch] -> CaveState (mins+1) coord equip
        _ -> transition (changeEquipment cs) coord

In [7]:
import Debug.Trace

toMeasure :: CaveState -> Measure CaveState
toMeasure cs@(CaveState m c t) = Measure (distanceFrom target c + (m * 2 `div` 3) - (if t == Torch then 7 else 0)) cs

improvement :: Set.Set CaveState -> CaveState -> Bool
improvement already (CaveState mins curr equip) = let
    similar = Set.filter (\(CaveState m c s) -> m <= mins && curr == c && s == equip) already
    in Set.null similar

search :: (CaveState -> Measure CaveState) -> Set.Set CaveState -> Set.Set (Measure CaveState) -> CaveState -> CaveState
search meas visited frontier state
    | current state == target && equipped state == Torch = state
    | otherwise = let
        visited' = Set.insert state visited
        possible = map meas $ filter (improvement visited') $ moves state
        ((Measure _ next), frontier') = Set.deleteFindMin $ Set.union frontier (Set.fromList possible)
        in search meas visited' frontier' (traceShowId next)

In [8]:
search toMeasure Set.empty Set.empty (CaveState 0 (0,0) Torch)

CaveState {minutesSoFar = 1027, current = (15,700), equipped = Torch}